In [79]:
import tweepy
import time
import logging
import random
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os
from os import environ
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
# Authenticating to to Twitter
API_KEY = "2RmSAmKyHHX2CBYSfCyjDvRHP"
API_SKEY = "kXFwdrkPnhBpYFgopXq053CUfLJsw4lGWj5rj4W0cmlt7SfduQ"
ACC_Token = "1353704693910921221-XmA1ZT6hcNlmyrlKFhqaQKvmQ4fd2T"
ACC_Token_Secret = "wJ7cogcpyZtwlMN4qmpPe9wQzFKVZUeo4wudRzExqKuaz"
auth = tweepy.OAuthHandler(API_KEY,API_SKEY)
auth.set_access_token(ACC_Token,ACC_Token_Secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
    wait_on_rate_limit_notify=True)
#Credential for firebase
cred = credentials.Certificate("twitter-threader-firebase-adminsdk-rh7d4-60df64a83c.json")

class userThread:
    def __init__(self,id, name,username,profile_img,tweets):
        self.id = id
        self.name = name
        self.username = username
        self.tweets = tweets
        self.profile_img = profile_img
    def to_dict(self):
        obj = {
            'id':self.id,
            'name':self.name,
            'username':self.username,
            'profile_img':self.profile_img,
            'tweets':{}
        }
        for tweet in self.tweets:
            tweet_id = tweet.tweet_id
            obj['tweets'][str(tweet_id)] = tweet.to_dict()
        return obj
class Tweet:
    def __init__(self, text,date,medias,tweet_id,urls):
        self.text = text
        self.date = date
        self.medias = medias
        self.tweet_id = tweet_id
        self.urls = urls ##contains list either empty or list with object url,expanded_url,display_url(sliced urls)
    def to_dict(self):
        obj = {
            'text':self.text,
            'date':self.date,
            'tweet_id':self.tweet_id,
            'urls':self.urls,
            'medias':self.medias
        }
        return obj
class ThreadCompiler:
    def __init__(self,tweet_id,user_id):
        self.tweet_id = tweet_id
        self.id = tweet_id ##storing parent id
        self.user_id = user_id
    def compileTweets(self):
        '''
        Compiles tweet of thread and return list of object of class type Tweet
        '''
        tweets = []
        parent_tweet_id = self.tweet_id
        print("ThreadCompiler: Fetching Tweets")
        while parent_tweet_id:
            tweet = api.get_status(parent_tweet_id, tweet_mode="extended")
            if tweet.user.id != self.user_id:
                break
            medias = []
            if 'media' in tweet.entities:
                media_entities = tweet.entities['media']
                for media_data in media_entities:
                    ##Currently only planning to save photo data as it'll be transformed to PDF
                    if media_data['type']=='photo':
                        medias.append(media_data['media_url_https'])
            tweetObj = Tweet(tweet.full_text,str(tweet.created_at),list(medias),parent_tweet_id,tweet.entities['urls'])
            tweets.append(tweetObj)
            parent_tweet_id = tweet.in_reply_to_status_id
        return list(tweets)
    def compileThread(self,tweets:"list of object type Tweet" = None):
        '''
        Compiles Thread of tweets and user and return object of class type userThread
        '''
        print("ThreadCompiler: Compiling Thread")
        if not tweets:
            tweets = self.compileTweets()
        tweets = tweets[::-1]
        self.id = tweets[0].tweet_id ##first id is used to save the thread
        user = api.get_user(self.user_id)
        return userThread(self.user_id,user.name,user.screen_name,user.profile_image_url_https,tweets)
    def save(self,threaData:"Dictionar"=None):
        '''
        threaData: Takes dictionary
        saves it to the firebase if not already exists and return True
        '''
        ##
        if not threaData:
            threaData = self.compileThread().to_dict()
        print("ThreadCompiler: Preparing to save!")
        FU = FirebaseUtility(cred)
        FU.storeData(self.id,threaData)
        return True
    def getThreadID(self):
        '''
        Returns the thread id
        Used to access documents
        '''
        return str(self.id)
class FirebaseUtility:
    def __init__(self,cred):
        self.cred = cred
        self.initialize() ##initialise first
        self.db = firestore.client()
    def initialize(self):
        try:
            if not firebase_admin._apps:
                print("FirebaseUtility:Initializing")
                firebase_admin.initialize_app(self.cred)
        except:
            print("FirebaseUtility:Error in cred FIX NEEDED!")
    def documentExists(self,thread_id:"Thread parent id",thread_len:"Length of fetched thread"):
        doc_ref = self.db.collection(u'threads').document(str(thread_id))
        doc = doc = doc_ref.get()
        if doc.exists:
            if len(doc.to_dict()['tweets']) < thread_len:
                print("FirebaseUtility:Thread Already Exists but shorter")
                return False
            else:
                print("FirebaseUtility:Thread Already Exists")
                return True
        else:
            print('FirebaseUtility:No such Thread!')
            return False
    def storeData(self,thread_id,data:"dictionary"):
        '''
        Stores the dictionary data with document name thread_id
        '''
        if not self.documentExists(thread_id,len(data['tweets'])):
            doc_ref = self.db.collection(u'threads').document(str(thread_id))
            doc_ref.set(data)
            print('FirebaseUtility:Thread {} Stored!'.format(str(thread_id)))
        else:
            pass
class ThreaderBot:
    def __init__(self,file_name="since_id.txt"):
        fread = open(file_name, 'r')
        self.since_id = int(fread.read().strip())
        fread.close()
    def retrieve_since_id(self,file_name="since_id.txt"):
        fread = open(file_name, 'r')
        since_id = int(fread.read().strip())
        fread.close()
        return since_id

    def store_since_id(self,since_id=None, file_name="since_id.txt"):
        if not since_id:
            since_id = self.since_id
        fwrite = open(file_name, 'w')
        fwrite.write(str(since_id))
        fwrite.close()
        return
    def fetchTweets(self):
        '''
        Fetches only mentioned tweets
        retweet will trigger this aswell
        '''
        mentions = api.mentions_timeline(self.since_id)
        mention = mentions[-1] if len(mentions) !=0 else None
        if mention:
            since_id = mention.id #Store the last id so that we can keep ourself updated
            self.store_since_id(since_id)
        return mentions
    def run(self):
        '''
        Returns unique list of recently mentioned tweets
        in_reply_to_status_id,in_reply_to_user_id
        Note:Twitter doesn't allow to tweet same tweet to same reply
        '''
        print("ThreaderBot: Running...")
        tweets = self.fetchTweets()
        if not tweets:
            print("ThreaderBot: Nothing New!")
            return False
        else:
            print("ThreaderBot: Threading...")
            tweet_ids = []
            for tweet in tweets:
                tweet_ids.append((tweet.in_reply_to_status_id,tweet.in_reply_to_user_id))
            tweet_ids = list(set(tweet_ids))
            return tweet_ids


In [80]:
bot = ThreaderBot()

In [82]:
runner_fetch = bot.run()
runner_fetch

ThreaderBot: Running...
ThreaderBot: Threading...


[(1289464819179876353, 797720168713441280)]

In [83]:
threadData = []
for tweet_id,user_id in runner_fetch:
    compiler = ThreadCompiler(tweet_id,user_id)
    if compiler.save():
        print(compiler.getThreadID())

ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Tweets
ThreadCompiler: Preparing to save!
FirebaseUtility:Thread Already Exists
1289444851868565504


In [78]:
threadData = []
for tweet_id,user_id in runner_fetch:
    compiler = ThreadCompiler(tweet_id,user_id)
    if compiler.save():
        print(compiler.getThreadID())

ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Tweets
ThreadCompiler: Preparing to save!
FirebaseUtility:Thread Already Exists
ThreadCompiler: Preparing to save!
1289444851868565504
ThreadCompiler: Compiling Thread
ThreadCompiler: Fetching Tweets
ThreadCompiler: Preparing to save!
FirebaseUtility:Thread Already Exists
ThreadCompiler: Preparing to save!
1354350890484813827


In [17]:
tweets = compiler.compileTweets()
len(tweets)

ThreadCompiler: Fetching Tweets


2

In [19]:
thread = compiler.compileThread(tweets)
thread.to_dict()

ThreadCompiler: Compiling Thread


{'id': 23061619,
 'name': 'Gul Panag',
 'username': 'GulPanag',
 'profile_img': 'https://pbs.twimg.com/profile_images/1336521973304709121/bHWQdCtx_normal.jpg',
 'tweets': {'1353996632073793536': {'text': 'The Tricolour CANNOT be disrespected.\nAbsolutely unacceptable.\nMust be condemned unequivocally.',
   'date': '2021-01-26 09:22:07',
   'tweet_id': 1353996632073793536,
   'urls': [],
   'medias': []},
  '1354013099527794689': {'text': 'Undesirable turn of events.\nI have supported this peaceful protest  from day one, however this violent turn is condemnable.\nAnd, on this sacred day, only the Tricolour 🇮🇳 should fly aloft the Red Fort.',
   'date': '2021-01-26 10:27:33',
   'tweet_id': 1354013099527794689,
   'urls': [],
   'medias': []}}}

In [44]:
len(thread.to_dict()['tweets'])

2

In [45]:
compiler.save(thread.to_dict())

ThreadCompiler: Preparing to save!


RetryError: Deadline of 120.0s exceeded while calling functools.partial(<function _wrap_unary_errors.<locals>.error_remapped_callable at 0x000001F1101B6DC0>, name: "projects/twitter-threader/databases/(default)/documents/threads/1354013099527794689"
, metadata=[('google-cloud-resource-prefix', 'projects/twitter-threader/databases/(default)'), ('x-goog-request-params', 'name=projects/twitter-threader/databases/%28default%29/documents/threads/1354013099527794689'), ('x-goog-api-client', 'gl-python/3.8.7 grpc/1.32.0 gax/1.25.1 gapic/2.0.2')]), last exception: 503 failed to connect to all addresses

In [35]:
doc = {'name': 'Gul Panag', 'username': 'GulPanag', 'profile_img': 'https://pbs.twimg.com/profile_images/1336521973304709121/bHWQdCtx_normal.jpg', 'id': 23061619, 'tweets': {'1354013099527794689': {'date': '2021-01-26 10:27:33', 'text': 'Undesirable turn of events.\nI have supported this peaceful protest  from day one, however this violent turn is condemnable.\nAnd, on this sacred day, only the Tricolour 🇮🇳 should fly aloft the Red Fort.', 'tweet_id': 1354013099527794689, 'medias': [], 'urls': []}, '1353996632073793536': {'text': 'The Tricolour CANNOT be disrespected.\nAbsolutely unacceptable.\nMust be condemned unequivocally.', 'urls': [], 'tweet_id': 1353996632073793536, 'medias': [], 'date': '2021-01-26 09:22:07'}}}
len(doc['tweets'])

2

In [66]:
li = [(23,24),(23,25),(22,29),(22,29)]
for x,y in li:
    print(x,y)

23 24
23 25
22 29
22 29


In [ ]:
compiler.save(thread.to_dict())